**IMPORTING NECESSARY LIBRARIES AND MODULES**

In [2]:
from bs4 import BeautifulSoup
import requests
import time
import smtplib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2

**FUNCTIONS TO PERFORM WEB-SCRAPING FROM THE AMAZON WEBPAGE**

In [3]:
#GETTING PRODUCT TITLES
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id":"productTitle"}).text.strip()
    except:
        title = "Unknown Product"
    
    return title      


In [4]:
#GETTING PRODUCT PRICES
def get_price(soup):
    try:
        price_whole = pd.to_numeric(soup.find("span", attrs={"class":"a-price-whole"}).text.strip())
        price_fraction = pd.to_numeric(soup.find("span", attrs={"class":"a-price-fraction"}).text.strip())
        price = price_whole + price_fraction
    except:
        price = "Unknown Price"
    
    return price

In [5]:
#GETTING PRODUCT RATINGS
def get_rating(soup):
    try:
        rating = pd.to_numeric(soup.find("span", attrs={"class":"a-icon-alt"}).text.strip().split()[0])
    except:
        rating = "Unknown Rating"
    
    return rating

In [6]:
#PERFORMING WEB SCRAPING WITH THE REQUESTS LIBRARY

URL = 'https://www.amazon.com/s?k=t-shirt&crid=1TZ2ZOV4HJN09&sprefix=%2Caps%2C186&ref=nb_sb_ss_recent_3_0_recent'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36","Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(URL, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")
links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

links_list = []

for link in links:
    links_list.append(link.get('href'))
    
data = {"title":[], "price":[], "rating":[]}

for link in links_list:
    product_links = "https://www.amazon.com" + link
    product_page = requests.get(product_links, headers=headers)
    product_soup = BeautifulSoup(product_page.content, "html.parser")
    data["title"].append(get_title(product_soup))
    data["price"].append(get_price(product_soup))
    data["rating"].append(get_rating(product_soup))

**LOADING DATAS INTO A PANDAS DATAFRAME, REMOVING UNKNOWN PRODUCTS**

In [7]:
df = pd.DataFrame.from_dict(data)
df['rating'] = df['rating'].replace('Unknown Rating', np.nan)
df['rating'] = df['rating'].astype(float)
df['price'] = df['price'].replace('Unknown Price', np.nan)
df['price'] = df['price'].astype(float)
df = df.dropna()
df = df.sort_values(by=['rating'], ascending=False)
df = df.reset_index(drop=True)
df

,title,price,rating
0,Nature Backs Enchanted (Charcoal) Nature Inspi...,29.0,5.0
1,LACOXA 2023 New Cotton T-Shirt Bitcoin 4D Digi...,75.0,5.0
2,Can't Hear You I'm Gaming Headset Graphic Vide...,118.0,4.7
3,INTO THE AM Mens T Shirt - Short Sleeve Crew N...,113.0,4.7
4,Carhartt Men's Loose Fit Heavyweight Short-Sle...,118.0,4.6
...,...,...,...
64,Sunborui Women's Summer Short Sleeve V Neck T ...,119.0,4.2
65,BILLIONHATS Wholesale Bulk 12 Pack Men's Cotto...,123.0,4.2
66,Men's Graphic Tees 100% Soft Cotton Crew Neck ...,122.0,4.1
67,Hanes Womens Originals V-Neck Long Sleeve T-Sh...,119.0,3.9


**LOADING DATA INTO A LOCAL POSTGRESQL DATABASE**

In [8]:
#CREATING CONNECTTION AND CURSOR TO POSTGRESQL DATABASE
def connect():
    def create_connection():
        conn = psycopg2.connect(
            host="localhost",
            database="database",
            user="postgres",
            password="Quockhanh2004@"
        )
        return conn
    conn = create_connection()
    cur = conn.cursor()
    return cur, conn
cur, conn = connect()

In [9]:
#PREPARE DATA TO INSERT INTO DATABASE
title = []
for item in data['title']:
    title.append(item)

price = []
for item in data['price']:
    price.append(item)

rating = []
for item in data['rating']:
    rating.append(item)

In [10]:
def load(data):
    cur, conn = connect()
    cur.execute("DROP TABLE IF EXISTS tshirts")
    cur.execute("CREATE TABLE tshirts (title TEXT, price FLOAT, rating FLOAT)")
    i = 0
    for i in range(len(title)):
        cur.execute("INSERT INTO tshirts (title, price, rating) VALUES (%s, %s, %s)", (title[i], price[i], rating[i]))
        i += 1
    conn.commit()
    print("Data loaded successfully")

database = load(data)

Data loaded successfully


**QUERYING DATA TO CHECK THE LOADING OF THE DATABASE**

In [13]:
def query():
    cur, conn = connect()
    cur.execute("SELECT * FROM tshirts")
    rows = cur.fetchall()
    for row in rows:
        print(f"Produc Title: {row[0]}, Price: {row[1]}, Rating: {row[2]} out of 5 stars")
    conn.close()

query()

Produc Title: Amazon Essentials Women's Classic-Fit Short-Sleeve Crewneck T-Shirt, Multipacks, Price: 19.0, Rating: 4.4 out of 5 stars
Produc Title: Gildan Men's Crew T-Shirts, Multipack, Style G1100, Price: 119.0, Rating: 4.6 out of 5 stars
Produc Title: Gildan Adult Ultra Cotton T-shirt, Style G2000, Multipack, Price: 119.0, Rating: 4.5 out of 5 stars
Produc Title: Fruit of the Loom Men's Eversoft Cotton Stay Tucked Crew T-Shirt, Price: 119.0, Rating: 4.6 out of 5 stars
Produc Title: Russell Athletic Men's Dri-Power Cotton Blend Tees & Tanks, Moisture Wicking, Odor Protection, UPF 30+, Sizes S-4X, Price: 119.0, Rating: 4.4 out of 5 stars
Produc Title: GAP Men's 3-Pack Everyday Short Sleeve Tee T-Shirt, Price: 119.0, Rating: 4.4 out of 5 stars
Produc Title: Hanes mens Beefyt T-shirt, Classic Heavyweight Cotton Crewneck Tee, Roomy Fit, 1 Or 2 Pack, Available in Tall, Price: 119.0, Rating: 4.4 out of 5 stars
Produc Title: Gildan Men's V-Neck T-Shirts, Multipack, Style G1103, Price: 119.